# HSE Summer School NLP&DA

![](https://www.hse.ru/data/2014/06/25/1309038576/logo_hse_cmyk_e.jpg)

Учебный проект по теме "Тональность отношений субъектов (именованных сущностей)", предполагающий разработку участниками школы под руководством тьюторов программных средств, решающих задачу определения мнения сторон о различных событиях, освещаемых в новостях

Нужен алгоритм, который по выделенным сущностям может найти пары этих сущностей в тексте и вырезать соответствующие куски между ними + несколько слов справа-слева. Возможно, имеет смысл идти от N-ой выделенной сущности до N+2, и включать всё до неё. 

Чем может помочь синтаксический анализ?

Чем поможет морфологический анализ?

In [1]:
import sys 
sys.path.append("/Users/dmitrys/anaconda2/lib/python2.7/site-packages/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Utility functions

In [74]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

def transformAnnotation(number):
    """
    Given number loads txt file with annotation 
    Returns DataFrame with transformed annotation
    """
    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    return d

def transformAnswer(number):
    """
    Given number loads txt file with answer 
    Returns DataFrame with transformed answer
    """
    answ = loadAnswer(number)
    answ = answ.split("\n")

    for i in range(len(answ)):
        answ[i] = answ[i].split(",")

    answ = pd.DataFrame(answ)
    answ.columns = ["entity_1", "entity_2", "attitude", "time"]
    answ.dropna(inplace=True)
    answ.time = answ.time.apply(lambda x: x.strip("\r"))
    
    return answ

def loadRuSentiLex():
    """
    Loads the RuSentiLex 2017 dictionary
    Returns data frame with ["word", "tag", "word_lemmatized", "tone", "certainty"]
    """
    with open("RuSentiLex2017_revised_2.txt") as f:
        Rusentilex = f.read().decode("cp1251").encode("utf8")
        Rusentilex = Rusentilex[1510:]

    Rusentilex = Rusentilex.split("\n")
    for i, item in enumerate(Rusentilex):
        Rusentilex[i] = item.split(",")

    for i in Rusentilex:
        if len(i) < 5:
            Rusentilex.remove(i)

    Rusentilex = pd.DataFrame(Rusentilex)
    Rusentilex.drop([5, 6, 7], axis=1, inplace=True)
    Rusentilex.columns = ["word", "tag", "word_lemmatized", "tone", "certainty"]
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip('\r'))
    
    return Rusentilex

![](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)

Словарь РуСентиЛекс

Структура: 
- 1 слово или словосочетание,
- 2 Часть речи или синтаксический тип группы,
- 3 слово или словосочетание в лемматизированной форме, 
- 4 Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная оценка, зависит от контекста (positive/negative),
- 5 Источник: оценка (opinion), чувство (feeling), факт (fact),
- 6 Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.

In [72]:
Rusentilex = loadRuSentiLex()

In [75]:
Rusentilex.head()

,word,tag,word_lemmatized,tone,certainty
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [77]:
d = transformAnnotation(2)
a = transformAnswer(2) 

In [78]:
d[d.entity!="Unknown"].head(10)

,entity,entity_car,pos_1,pos_2
0,Author,PER,1,7
2,Author,PER,22,28
4,Россия,LOC,39,45
5,Сирии,LOC,72,77
6,Author,PER,81,87
8,Author,PER,147,153
10,Россия,LOC,164,170
11,Сирии,LOC,195,200
12,Author,PER,249,255
14,Author,PER,299,305


Для случая трёх сущностей, где первая относится к третьей, но не относится ко второй - "Россия провела встречу во Франции, чтобы осудить действия Сирии" - можно брать для оценки мнения только слова, идущие после второй сущности

In [90]:
a.head()

,entity_1,entity_2,attitude,time
0,Сирия,Россия,pos,current
1,США,Асад,neg,past
2,Москва,Израиль,pos,current
3,Израиль,Россия,pos,current
4,Сирия,Израиль,neg,past


In [82]:
from nltk.tokenize import wordpunct_tokenize

In [89]:
x = word_tokenize("Россия реализовала свои цели в Сирии, по крайней мере в краткосрочной перспективе. ")
for i in x:
    print(i)

Россия
реализовала
свои
цели
в
Сирии
,
по
крайней
мере
в
краткосрочной
перспективе
.


In [21]:
t = loadText(2)

In [23]:
print(t[:500])

{Author, Unknown} 

{Author, Unknown} Россия и кратковременный успех в Сирии

{Author, Unknown} inosmi.ruПосмотреть оригиналапрель 5-го, 2016

{Author, Unknown} Россия реализовала свои цели в Сирии, по крайней мере в краткосрочной перспективе. 
{Author, Unknown} Что будет дальше, пока неясно. 
{Author, Unknown} Точно можно сказа


In [34]:
print(t.split("{Author, Unknown}")[6])

 Точно можно сказать одно: подавляющее большинство сирийских граждан не смогут забыть российского участия в войне, которая была направлена против всего сирийского народа.





In [20]:
from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [15]:
for i in range(10):
    print(lemmatize(t[i]))

{Author,
Unknown}
{Author,
Unknown}
россия
и
кратковременный
успех
в
сирия
